In [6]:
from dotenv import load_dotenv
import os
from glob import glob
import mne
import numpy as np
import torch
import torch.nn as nn
import gc 
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

load_dotenv()
root_dir = os.getenv("ROOT_DIR")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Model definitions
class InnerSpeechDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [4]:
# CNN/LSTM hybrid
class InnerSpeechModel(nn.Module):
    def __init__(self):
        super().__init__()

        # CNN component: outputs 256 channels
        self.convolv = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1),  # Fixed to 256 channels
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3)
        )

        # Bi-LSTM component (2 Layers)
        self.lstm = nn.LSTM(input_size=256, hidden_size=128, num_layers=2, batch_first=True, bidirectional=True)

        self.attn_weight = nn.Linear(2 * 128, 1, bias=False)

        # Fully connected layer
        self.fc = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(2*128, 4)  # Matches hidden_size=128
        )

    def forward(self, x):
        # Input shape: (batch, 20, 64)
        x = x.permute(0, 2, 1)  # Shape: (batch, 64, 20)
        x = self.convolv(x)      # Shape: (batch, 256, 20)
        x = x.permute(0, 2, 1)   # Shape: (batch, 20, 256)

        lstm_out, (h_n, c_n) = self.lstm(x)  # lstm_out shape: (batch, 20, 128)

        # Compute attention scores
        # Flatten across features: attn_score[i, t] = wT * h_{i, t}
        # Then softmax over t to get α_{i, t}
        attn_scores = self.attn_weight(lstm_out).squeeze(-1)
        attn_weights = torch.softmax(attn_scores, dim=1)
        # Weighted sum of LSTM outputs:
        attn_applied = torch.bmm(attn_weights.unsqueeze(1), lstm_out).squeeze(1)

        # Regression to 3D motion
        output = self.fc(attn_applied)
        return output

### Initial Analysis

In [7]:
data_l_epochs = sorted(glob(os.path.join(root_dir, "**", 'sub-*_ses-*_eeg-epo.fif'), recursive=True))

In [8]:
data_l_epochs = [data for data in data_l_epochs if "ses-03" not in data]

In [ ]:
X_all = []
y_all = []
for data in data_l_epochs:
    epochs = mne.read_epochs(data, preload=True)
    X = epochs.get_data()
    event_map = {v: k for k, v in epochs.event_id.items()}
    y = [event_map[event[-1]] for event in epochs.events]
    X = torch.tensor(X, device=device)

    X_all.append(X)
    y_all.append(y)

In [ ]:
# Saving Labels

# y_all = np.array(y_all)
# unique_labels = sorted(set(y_all.flatten()))
# label_mapping = {label: index for index, label in enumerate(unique_labels)}
# int_labels = np.vectorize(label_mapping.get)(y_all)
# tensor_labels = torch.tensor(int_labels, dtype=torch.long, device=device)
# torch.save(tensor_labels,"data/labels.pth")

In [ ]:
# Loading the Labels
tensor_labels = torch.load("data/labels.pth")

In [ ]:
X_train = X_all[0:15]

In [ ]:
X_test = X_all[15:]

In [ ]:
X_test = torch.concat(X_test, dim=0)

In [ ]:
# 15: 20
X_test.shape

In [ ]:
y_test = tensor_labels[15:]

In [ ]:
torch.save(y_test, "data/y_test.pth")
torch.save(X_test, "data/X_test.pth")

In [ ]:
del y_test

In [ ]:
del X_test

In [ ]:
gc.collect()

In [ ]:
X_train = torch.concat(X_train, dim = 0)

In [ ]:
torch.save(X_train, "data/X_train.pth")

In [ ]:
y_train = tensor_labels[:15]

In [ ]:
torch.save(y_train, "data/y_train.pth")

In [ ]:
# Future Progess:
# Load the data into a Neural Network
# Train the Neural Network on the data
# Output predictions from the neural network

# Generate synthetic neural signal based on this data
# stream the synthetic neural signal to chat-studio using websockets
# use an api with the trained neural network to decode the neural signal 
# display the neural signal as a suggested selection for text input

## Training a Neural Network

In [5]:
X_train = torch.load("data/X_train.pth")
y_train = torch.load("data/y_train.pth")
X_test = torch.load("data/X_test.pth")
y_test = torch.load("data/y_test.pth")

/tmp/ipykernel_47751/3539107226.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_train = torch.load("data/X_train.pth")
/tmp/ipykernel_47751/3539107226.py:2: FutureWarn

In [ ]:
y_train = y_train.cpu().numpy()

In [ ]:
y_train = y_train.cpu().numpy()
X_train = X_train.cpu().numpy()

### Model Definition

In [ ]:
X_train.device

In [ ]:
X_train_split, X_val_split, y_train_split, y_validation_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

In [ ]:
train_loader = InnerSpeechDataset(X_train_split, y_train_split)
val_loader = InnerSpeechDataset(X_val_split, y_validation_split)

In [ ]:
train_loader = InnerSpeechDataset()